# Natural Language Processing. Основные понятия

*Паршина Анастасия, НИУ ВШЭ*

Сегодня мы с вами познакомимся с основными понятиями обработки естественного языка и разберем применение самых простых инструментов на небольших примерах. В дальнейшем все это нам сильно пригодится. Мы поговорим про понятия стемминга и лемматизации, также посмотрим на формирование n-грамм, узнаем, что такое стоп-слова, и посмотрим, как эти инструменты работают с русскоязычными текстами. 

## Стемминг и лемматизация

### Стемминг

Для стемминга нам понадобится импортировать модуль `nltk`, в котором есть реализация стеммера Портера. 

*Важно! Вероятро, при первом запуске команды `import nltk` у вас возникнет ошибка `ModuleNotFoundError: No module named 'nltk'`. Один из вариантов ее решения: запустить команду `!pip install nltk`, она написана для вас в виде комментария после знака `#`.*

In [1]:
#!pip install nltk
import nltk

Далее нам предстоит долгий путь. Обратимся к модулю `nltk`, затем к пакету `stem` из него и модулю `porter`. В конце этой истории нам нужен класс `PorterStemmer()`.

Вы можете вызвать команду `help(nltk.stem.porter)`, чтобы посмотреть подробную документацию, в частности — описание самого алгоритма стемминга Портера.

У самого класса есть метод `.stem()`. Его можно применить так: 

In [2]:
nltk.stem.porter.PorterStemmer().stem('argued')

'argu'

Но эта конструкция очень длинная, поэтому часть `nltk.stem.porter.PorterStemmer()` мы можем записать в отдельную перемнную, например, назовем ее `stemmer`. Затем уже можно применять метод `.stem()` к этой переменной. 

In [32]:
stemmer = nltk.stem.porter.PorterStemmer()

print(stemmer.stem('argue'))
print(stemmer.stem('argued'))
print(stemmer.stem('arguing'))

argu
argu
argu


Обратите внимание на пример из презентации. Вот его наглядная реализация в Python.

Если мы обратим внимание на документацию, то увидим, что в методе `.stem()` проставлен параметр `to_lowercase=True`, то есть слова автоматически приводятся к нижнему регистру (строчными буквами). 

In [4]:
print(stemmer.stem('ArGue'))

argu


Также важно, что метод работает только с одним словом, ему нельзя передать сразу несколько, например, в виде списка. Будет выдана ошибка `AttributeError: 'list' object has no attribute 'lower'`, то есть к списку нельзя применить метод `.lower()`, потому что это метод строк.

In [5]:
print(stemmer.stem(['ArGue', 'argue']))

AttributeError: 'list' object has no attribute 'lower'

Однако мы можем обратиться сразу к нескольким словам в цикле `for`.

In [33]:
words = ('argue', 'argued', 'arguing')

for word in words:
    print(stemmer.stem(word))

argu
argu
argu


Исходный кортеж `words` в таком случае не изменился. Если мы хотим не просто напечатать результат, а сохранить его куда-то, то можно для этого использовать список и метод списков `.append()`.

In [34]:
stem_words = [] # Пустой список

for word in words:
    stem_words.append(stemmer.stem(word)) # Добавляем основу слова в список
    
print(stem_words) 
# Если нам нужны только уникальные значения, то можно использовать множество и метод .add()

['argu', 'argu', 'argu']


Быстрее будут работать списковые включения (list comprehensions), которые можно реализовать так:

In [35]:
stem_words2 = [stemmer.stem(word) for word in words]
print(stem_words2)

['argu', 'argu', 'argu']


### Лемматизация

Далее посмотрим, как работает лемматизация в Python.

Аналогично стеммингу мы обращаемся к классу `WordNetLemmatizer()` и его методу `.lemmatize()`. Единственно отличие в том, что при запуске будет выдаваться ошибка `LookupError: Resource wordnet not found.`. Чтобы ее решить нужно запустить команду `nltk.download('wordnet')`.

Можете ознакомиться с документацией, запустив команду `help(nltk.stem.wordnet)`.

In [36]:
#nltk.download('wordnet')
lemm = nltk.stem.wordnet.WordNetLemmatizer()

print(lemm.lemmatize('argue'))
print(lemm.lemmatize('argued'))
print(lemm.lemmatize('arguing'))

argue
argued
arguing


Получили что-то странное, отличное от информации из презентации. На самом деле, у метода `.lemmatize()` есть параметр `pos`, то есть `Part of Speech` (часть речи), значение которого по умолчанию равно `n`, то есть noun (существительное).

    "n" — nouns
    "v" — verbs
    "a" — adjectives
    "r" — adverbs 
    "s" — satellite adjectives

Дабы сделать лемматизацию более точной, мы можем указать, с какой частью речи мы работаем. 

Определение частей речи зашито в модуль `textblob`, из него нам нужен класс `TextBlob`.

In [ ]:
#!pip install textblob
from textblob import TextBlob

Передадим ему наши слова в виде строки и вызовем атрибут `tags`, который выдаст нам список кортежей со словами и их частями речи.

В нашем случае `NN` обозначает `Noun, singular or mass` (существительное), `VBD` — `Verb, past tense` (глагол прошедшего времени), `VBG` — `Verb, gerund or present participle` (глагол, герундий или причастие настоящего времени). Конечно, это все относится к английскому языку.

Если ячейка выдает ошибку, то необходимо запустить первые две команды, убрав знак `#`.

In [53]:
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
text_blob_object = TextBlob(('argue argued arguing'))
print(text_blob_object.tags)

[('argue', 'NN'), ('argued', 'VBD'), ('arguing', 'VBG')]


Теперь попробуем провести лемматизацию для этих слов. 

Обратимся к каждому кортежу в нашем списке. Первый (нулевой по индексу) элемент кортежа — это наше слово. А второй (первый по индексу) — это часть речи. Однако, вы уже могли обратить внимание, что параметр `pos` принимает определенные значения (выписаны выше), значит, например, `NN` должно быть преобразовано в `n`, `VBD` — в `v` и т.д. На нашем простом примере достаточно забрать первый элемент строки, например, `N` из `NN` и привести к нижнему регистру. 

In [54]:
words = text_blob_object.tags

for word in words:
    print(word)
    print(lemm.lemmatize(word[0], pos=word[1][0].lower()))

('argue', 'NN')
argue
('argued', 'VBD')
argue
('arguing', 'VBG')
argue


## N-граммы

Это комбинации слов, стоящих рядом в предложениях. Метод, который позволяет получить N-граммы также реализован для класса `TextBlob` и называется он `.ngrams()`. На вход он принимает один параметр — целое число, обозначающее `N`.

In [60]:
# N = 1
print(text_blob_object.ngrams(1))

# N = 2
print(text_blob_object.ngrams(2))

# N = 3
print(text_blob_object.ngrams(3))

# и т.д.

[WordList(['argue']), WordList(['argued']), WordList(['arguing'])]
[WordList(['argue', 'argued']), WordList(['argued', 'arguing'])]
[WordList(['argue', 'argued', 'arguing'])]


Разберем пример из презентации со строкой `"I love Python very much"`.

In [61]:
phrase_eng = "I love Python very much"
text_blob_object = TextBlob(phrase_eng)

# Вот, например, мы получили биграммы
print(text_blob_object.ngrams(2))

[WordList(['I', 'love']), WordList(['love', 'Python']), WordList(['Python', 'very']), WordList(['very', 'much'])]


Предположим, мы хотим получить вообще все возможные N-граммы фразы. Следовательно, максимальное значение `N` будет равно количеству слов в предлежении. 

Все количество слов можно узнать, обратившийсь к атрибуту `words`. Количество слов можно узнать, применив функцию `len()`.

In [64]:
print(len(text_blob_object.words)) # максимальное N = 5

5


Нам нужно сгенерировать значения N от 1 до 5 включительно. Для этого воспользуемся функцией `range()` с такими параметрами: `range(1, len(text_blob_object.words) + 1)`. Напомним, что значение `stop` функция `range()` не включает, поэтом дополнительно прибавляем один.

In [70]:
for N in range(1, len(text_blob_object.words) + 1):
    print(f'N = {N}')
    print(text_blob_object.ngrams(N))

N = 1
[WordList(['I']), WordList(['love']), WordList(['Python']), WordList(['very']), WordList(['much'])]
N = 2
[WordList(['I', 'love']), WordList(['love', 'Python']), WordList(['Python', 'very']), WordList(['very', 'much'])]
N = 3
[WordList(['I', 'love', 'Python']), WordList(['love', 'Python', 'very']), WordList(['Python', 'very', 'much'])]
N = 4
[WordList(['I', 'love', 'Python', 'very']), WordList(['love', 'Python', 'very', 'much'])]
N = 5
[WordList(['I', 'love', 'Python', 'very', 'much'])]


Можем также распаковать результат работы метода `.ngrams()`:

In [72]:
for N in range(1, len(text_blob_object.words) + 1):
    print(f'N = {N}')
    for ngram in text_blob_object.ngrams(N):
        print(*ngram) # Знак * в функции print() отвечает за распаковку списка

N = 1
I
love
Python
very
much
N = 2
I love
love Python
Python very
very much
N = 3
I love Python
love Python very
Python very much
N = 4
I love Python very
love Python very much
N = 5
I love Python very much


## Стоп-слова и их удаление

Стоп-слова — это своеобразный шум, который мешает при обработке даных, например, предлоги, союзы, междометия, поэтому их часто убирают. Давайте посмотрим на стоп-слова в английском языке. 

Нам все также понадобится модуль `nltk`. Предварительно запустите команду `nltk.download("stopwords")`.

In [77]:
#nltk.download("stopwords") 
stop = nltk.corpus.stopwords
print(stop.words("english")) # Все стоп-слова английского языка

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Обратите внимание, что это список. 

Попробуем убрать стоп-слова из предложения. 

In [96]:
text = 'My God! Your eyes are as beautiful as the sky, your soul is reflected in them. \
It is pure and immaculate as the soul of a baby. My lovely beautiful soul! I am in love with you!'

# Обратите внимание на символ \ — он позволяет перенести код на седующую строку и не мешает его работе

print(text)

My God! Your eyes are as beautiful as the sky, your soul is reflected in them. It is pure and immaculate as the soul of a baby. My lovely beautiful soul! I am in love with you!


Сначала соберем все слова из нашего текста с помощью класса `TextBlob` и атрибута `words`. Предватительно все слова в тексте приведем к нижнему регистру с помощью метода `.lower()`.

In [97]:
text_blob_object = TextBlob(text.lower())
print(text_blob_object.words)

['my', 'god', 'your', 'eyes', 'are', 'as', 'beautiful', 'as', 'the', 'sky', 'your', 'soul', 'is', 'reflected', 'in', 'them', 'it', 'is', 'pure', 'and', 'immaculate', 'as', 'the', 'soul', 'of', 'a', 'baby', 'my', 'lovely', 'beautiful', 'soul', 'i', 'am', 'in', 'love', 'with', 'you']


Далее вариантов работы несколько. Например, если нам не важен порядок слов, то мы можем использовать разность множеств, то есть 

    1. Все слова нашего такста сделать множество 
    2. Преобразовать список со стоп-словами во множество 
    3. Из первого множества вычесть второе (или использовать метод .difference())
    
Напомним, что в множествах элементы уникальные.

In [98]:
print(set(text_blob_object.words) - set(stop.words("english")))
print(set(text_blob_object.words).difference(set(stop.words("english"))))

{'sky', 'beautiful', 'baby', 'lovely', 'immaculate', 'soul', 'god', 'reflected', 'eyes', 'pure', 'love'}
{'sky', 'beautiful', 'baby', 'lovely', 'immaculate', 'soul', 'god', 'reflected', 'eyes', 'pure', 'love'}


Если же нам важно сохранить порядок слов, то можно использовать цикл `for`.

In [99]:
result = [] # Пустой список

for word in text_blob_object.words:        # Обращаемся к каждому слову в тексте
    if word not in stop.words("english"):  # Если слова нет в стоп-словах, то 
        result.append(word)                # добавляем его в список
        
print(result)

['god', 'eyes', 'beautiful', 'sky', 'soul', 'reflected', 'pure', 'immaculate', 'soul', 'baby', 'lovely', 'beautiful', 'soul', 'love']


Аналогично можно использовать списковые включения:

In [100]:
result2 = [word for word in text_blob_object.words if word not in stop.words("english")]
print(result2)

['god', 'eyes', 'beautiful', 'sky', 'soul', 'reflected', 'pure', 'immaculate', 'soul', 'baby', 'lovely', 'beautiful', 'soul', 'love']


## Работа с русским языком

Теперь проверим, как все вышеперечисленное работает с русским языком. Начнем со стеммера Портера. 

### Стемминг

Используем тот алгоритм, что и выше.

In [102]:
print(stemmer.stem('бежать'))
print(stemmer.stem('бегущий'))
print(stemmer.stem('бегающий'))

бежать
бегущий
бегающий


Почему-то ничего не изменилось. Это связано с тем, что оригинальный стеммер Портера работает только с английским языком. Однако, данный алгоритм был доработан и для других языков. 

Мы все еще работаем с модулем `nltk`, обращаемся к классу `RussianStemmer()`.

Если посмотреть документацию с помощью команды `help(nltk.stem.snowball)`, то можно увидеть и другие языки, для которых реализован алгоритм.

In [104]:
rus_stemmer =  nltk.stem.snowball.RussianStemmer()

print(rus_stemmer.stem('бежать'))
print(rus_stemmer.stem('бегущий'))
print(rus_stemmer.stem('бегающий'))

бежа
бегущ
бега


Однако, мы видим, что стемминг не всегда точно работает с русским языком, и лучше будет использовать лемматизацию.

### Лемматизация

Аналогично посмотрим, как работает метод `.lemmatize()` с русскими словами.

In [106]:
print(lemm.lemmatize('бежать'))
print(lemm.lemmatize('бегущий'))
print(lemm.lemmatize('бегающий'))

бежать
бегущий
бегающий


И снова не работает! Для работы и лемматизации русских слов комфортнее использовать модуль `pymorphy3`. При первом запуске вам необходимо будет запустить команду `!pip install pymorphy3`.

In [107]:
#!pip install pymorphy3
import pymorphy3

Доберемся до класса `MorphAnalyzer()` и используем его метод `.parse()`.

In [138]:
morph = pymorphy3.analyzer.MorphAnalyzer()

for data in morph.parse('бежать'):
    print(data)

Parse(word='бежать', tag=OpencorporaTag('INFN,perf,intr'), normal_form='бежать', score=0.5, methods_stack=((DictionaryAnalyzer(), 'бежать', 392, 0),))
Parse(word='бежать', tag=OpencorporaTag('INFN,impf,intr'), normal_form='бежать', score=0.5, methods_stack=((DictionaryAnalyzer(), 'бежать', 392, 42),))


Давайте разберемся, почему было выдано два варианта и что они вообще означают.

Нам нужно обратить внимание на `tag` в обоих случаях.

In [140]:
print(morph.parse('бежать')[0].tag)
print(morph.parse('бежать')[1].tag)

INFN,perf,intr
INFN,impf,intr


In [137]:
p = morph.parse('бегущий')[0]
p.tag.transitivity

'intr'

Разберемся, что означает каждый из них:
    
    INFN,perf,intr — глагол инфинитив, совершенного вида, непереходный
    INFN,impf,intr — глагол инфинитив, несовершенного вида, непереходный
    
*Небольшие пояснения:*

+ Совершенный вид глагода отвечает на вопрос "что сделать?". Например, "Они бросились врассыпную, пытаясь бежать от служителей порядка". 

+ Несовершенный вид глагола отвечает на вопрос "что делать?". Например, "Чтобы спастись, нужно бежать далеко и быстро".

+ Непереходные глаголы обозначают действия, не направленные на какой-либо предмет. Например, в нашем случае "Чтобы спастись, нужно бежать далеко и быстро".

+ Переходные глаголы — глаголы, действие которых направлено на другой предмет. Например, "Нужно хорошо подготовиться, чтобы бежать дистанцию". Обратите внимание, что даже `MorphAnalyzer()` не все знает про русские слова, однако, многое он определяет достаточно точно.
    
С обозначениями можно ознакомиться на официальном сайте с [документацией](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs).

Также нам может быть интересно значение атрибута `score`. 

In [142]:
print(morph.parse('бежать')[0].score)
print(morph.parse('бежать')[1].score)

0.5
0.5


В нашем случае оба равны `0.5`, то есть в равной степени может существовать как первая интерпретация глагола, так и вторая.

Посмотрим на слово `"были"`. 

In [147]:
for data in morph.parse('были'):
    print(data)

Parse(word='были', tag=OpencorporaTag('VERB,impf,intr plur,past,indc'), normal_form='быть', score=0.997032, methods_stack=((DictionaryAnalyzer(), 'были', 620, 7),))
Parse(word='были', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='быль', score=0.000593, methods_stack=((DictionaryAnalyzer(), 'были', 13, 1),))
Parse(word='были', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='быль', score=0.000593, methods_stack=((DictionaryAnalyzer(), 'были', 13, 2),))
Parse(word='были', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='быль', score=0.000593, methods_stack=((DictionaryAnalyzer(), 'были', 13, 5),))
Parse(word='были', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='быль', score=0.000593, methods_stack=((DictionaryAnalyzer(), 'были', 13, 6),))
Parse(word='были', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='быль', score=0.000593, methods_stack=((DictionaryAnalyzer(), 'были', 13, 9),))


Здесь вариантов уже больше. Наиболее вероятный — первый, со значением `score=0.997032`, то есть с большой вероятностью мы имеем дело с глаголом прошедшего времени, нормальная форма которого `быть`. Однако это может быть и множественное число слова `быль`, хотя и с меньшей вероятностью.

Суммарно значение `score` должно быть равно 1, однако из-за особенностей работы с вещественными числами, оно всегда будет около `0.999999`.

Чтобы забрать нормальную форму слова, нужно обратиться к ней с помощью атрибута `normal_form`. 

In [150]:
morph.parse('были')[0].normal_form

'быть'

### N-граммы

А вот работа с N-граммами не меняется, все остается также, как и было.

In [151]:
phrase_rus = "Я очень сильно люблю Python"
text_blob_object = TextBlob(phrase_rus)

# Вот, например, мы получили биграммы
print(text_blob_object.ngrams(2))

[WordList(['Я', 'очень']), WordList(['очень', 'сильно']), WordList(['сильно', 'люблю']), WordList(['люблю', 'Python'])]


Аналогично можно посмотреть вообще все N-граммы.

In [152]:
for N in range(1, len(text_blob_object.words) + 1):
    print(f'N = {N}')
    for ngram in text_blob_object.ngrams(N):
        print(*ngram) # Знак * в функции print() отвечает за распаковку списка

N = 1
Я
очень
сильно
люблю
Python
N = 2
Я очень
очень сильно
сильно люблю
люблю Python
N = 3
Я очень сильно
очень сильно люблю
сильно люблю Python
N = 4
Я очень сильно люблю
очень сильно люблю Python
N = 5
Я очень сильно люблю Python


### Удаление стоп-слов

Очевидно, что в русском языке свой набор стоп-слов. Вызвать его можно также с помощью модуля `nltk`.

In [153]:
print(stop.words("russian")) # Все стоп-слова русского языка

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Логика удаления стоп-слов не меняется. 

In [154]:
text_rus = 'Боже мой! Твои глаза прекрасны, как небо, в них отражается твоя душа. \
Она чиста и непорочна, как душа ребенка. Моя милая, прекрасная душа! Я влюблен в тебя!'

# Обратите внимание на символ \ — он позволяет перенести код на седующую строку и не мешает его работе

print(text_rus)

Боже мой! Твои глаза прекрасны, как небо, в них отражается твоя душа. Она чиста и непорочна, как душа ребенка. Моя милая, прекрасная душа! Я влюблен в тебя!


In [157]:
text_blob_object = TextBlob(text_rus.lower())
print(text_blob_object.words)

['боже', 'мой', 'твои', 'глаза', 'прекрасны', 'как', 'небо', 'в', 'них', 'отражается', 'твоя', 'душа', 'она', 'чиста', 'и', 'непорочна', 'как', 'душа', 'ребенка', 'моя', 'милая', 'прекрасная', 'душа', 'я', 'влюблен', 'в', 'тебя']


In [158]:
result_rus = [word for word in text_blob_object.words if word not in stop.words("russian")]
print(result_rus)

['боже', 'твои', 'глаза', 'прекрасны', 'небо', 'отражается', 'твоя', 'душа', 'чиста', 'непорочна', 'душа', 'ребенка', 'милая', 'прекрасная', 'душа', 'влюблен']
